<a href="https://colab.research.google.com/github/utsavajay/projects/blob/main/Cats_vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download  dogs-vs-cats-redux-kernels-edition

 99% 807M/814M [00:08<00:00, 86.1MB/s]
100% 814M/814M [00:09<00:00, 94.2MB/s]


In [ ]:
!unzip dogs-vs-cats-redux-kernels-edition.zip
!unzip test.zip
!unzip train.zip 

In [ ]:
import os, cv2, re, random
import numpy as np
import pandas as pd
import re
import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models, optimizers
from keras import backend as K
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
TRAIN_DIR = '../content/train/'
TEST_DIR = '../content/test'

train = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] 
test = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [ ]:
## Subsampling for both dogs and cats

sampled_set = []

dog_ct = 0
cat_ct = 0

for item in train: 
  if len(re.findall('dog',item)) != 0 and dog_ct < 1500:
      sampled_set.append(item)
      dog_ct+=1
  elif cat_ct <1500:
      sampled_set.append(item)
      cat_ct+=1


In [ ]:
size = (300, 300)

X = [(cv2.resize(cv2.imread(image), size, interpolation=cv2.INTER_CUBIC)) for image in sampled_set]
y = [1 if 'dog' in item else 0 for item in X]

<ipython-input-11-effd20524387>:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  y = [1 if 'dog' in item else 0 for item in X]


In [ ]:
print(len(X))
print(len(y))


3000
3000


In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X,y, test_size=0.2, random_state=1)
nb_train = len(X_train)
nb_validation = len(X_val)
batch_size = 25

In [ ]:
print(nb_train)
print(nb_validation)

2400
600


In [ ]:
def build_model():

    model = Sequential()
      
    
    model.add(layers.Input(shape=(300, 300, 3)))
    model.add(img_aug)
      

    base_model = EfficientNetB3(include_top=False, weights="imagenet")
    base_model.trainable = False
    model.add(base_model)

    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    

    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(2, activation="softmax"))


    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])


    return model


In [ ]:
classifier = build_model()

43941136/43941136 [==============================] - 0s 0us/step


In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_aug (Sequential)        (None, 300, 300, 3)       0         
                                                                 
 efficientnetb3 (Functional)  (None, None, None, 1536)  10783535 
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1536)             6144      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 2)                 3

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [ ]:
history = classifier.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=nb_validation // batch_size
)